Azure AI Search with LlamaIndex

This code demonstrates how to use Azure AI Search with Azure OpenAI and the LlamaIndex data framework

Install packages

In [2]:
! pip install -r requirements.txt --quiet

You should consider upgrading via the '/Users/sithukaungset/Azure-AI-Search-prompthon/venv/bin/python3 -m pip install --upgrade pip' command.


Load .env file

In [4]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Make sure your .env file has values for the following environment variables
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# Llama Index does not support RBAC authentication, an API key is required
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
if len(azure_openai_key) == 0:
    raise Exception("API key required")
azure_openai_embedding_model = os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"]
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_chatgpt_deployment = os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
embedding_dimensions = os.environ["AZURE_OPENAI_EMBEDDING_DIMENSIONS"]

Configure an embeddings instance

In [5]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI

embeddings = AzureOpenAIEmbedding(
    model_name=azure_openai_embedding_model,
    deployment_name=azure_openai_embedding_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

llm = AzureOpenAI(
    deployment_name=azure_openai_chatgpt_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key
)

Upload, vectorize, and index documents

This step reads PDFs from a local folder, calls the embedding model for vectorization, and then calls a search client to index the content on Azure AI Search.



In [6]:

from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement, MetadataIndexFieldType
from llama_index.core import StorageContext, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from azure.search.documents.indexes import SearchIndexClient

metadata_fields = {
    "author": "author",
    "theme": ("topic", MetadataIndexFieldType.STRING),
    "director": "director",
}

vector_store = AzureAISearchVectorStore(  
    search_or_index_client=SearchIndexClient(endpoint=endpoint, credential=credential),  
    filterable_metadata_field_keys=metadata_fields,
    index_name=index_name,  
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,  
    id_field_key="id",  
    chunk_field_key="content",  
    embedding_field_key="content_vector",  
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    embedding_dimensionality=embedding_dimensions,
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
Settings.llm = llm
Settings.embed_model = embeddings
directory = os.path.abspath(os.path.join("..", "data", "documents"))
documents = SimpleDirectoryReader(directory).load_data()
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context)

ValueError: Directory /Users/data/documents does not exist.

Perform a vector similarity search


In [ ]:
query_engine = index.as_query_engine(llm)
response = query_engine.query("What is included in my Northwind Health Plus plan that is not in standard?")
print(response)